# Barotropic/clinic gyre simulation with Oceananigans.jl

credits to *Simone Silvestri* and everyone else from the Oceananigans team!

The notebook uses
- Oceananigans v0.98
- Julia 1.10 (not 1.11!)

### Installation instructions

To install a Jupyter kernel for Julia 1.10

```julia
using Pkg
Pkg.add("IJulia")
using IJulia
installkernel("Julia")

```

then in Jupyter change kernel to "Julia 1.10.10" (v1.10 with current patch version 10).
Packages required

```julia
using Pkg
Pkg.add("Oceananigans")   # the ocean model
Pkg.add("GLMakie")        # for visualisation (let's make a video!)
Pkg.add("JLD2")           # for data output (alternative to netCDF)
```

then you are ready to run this notebook!


In [ ]:
using Oceananigans

In [ ]:
using Oceananigans.Grids
using Oceananigans.Advection: EnergyConserving, EnstrophyConserving
using Oceananigans.Units
using Statistics
using Printf

# Problem: Wind-driven upper ocean circulation in the North Atlantic

From the Oceananigns [documentation](https://clima.github.io/OceananigansDocumentation/stable/physics/hydrostatic_free_surface_model/): Let's use the hydrostatic equations

$$
\begin{align}
\partial_t \boldsymbol{u} & = - \left ( \boldsymbol{u} \boldsymbol{\cdot} \boldsymbol{\nabla} \right ) \boldsymbol{u}
                        - \boldsymbol{f} \times \boldsymbol{u}
                        - \boldsymbol{\nabla}_h (p + g \eta)
                        - \boldsymbol{\nabla} \boldsymbol{\cdot} \boldsymbol{\tau}
                        + \boldsymbol{F_u} \\
0 & = b - \partial_z p \\
\partial_t \eta &= w(x, y, z=0, t) \\
0 &= \boldsymbol{\nabla}_h \boldsymbol{\cdot} \boldsymbol{u} + \partial_z w \\
\end{align}
$$

where $b$ the is buoyancy, $\boldsymbol{\tau}$ is the hydrostatic kinematic stress tensor,
$\boldsymbol{F_u}$ denotes an internal forcing of the horizontal flow $\boldsymbol{u}$,
$\boldsymbol{v} = \boldsymbol{u} + w \hat{\boldsymbol{z}}$ is the three-dimensional flow,
$p$ is kinematic pressure, $\eta$ is the free-surface displacement, and $\boldsymbol{f}$
is the *Coriolis parameter*, or the background vorticity associated with the specified rate of
rotation of the frame of reference.

We want to solve this in a rectangular box (for simplicity) centred on the North Atlantic, something like this

(reference to videos shown in presentation)

with the windstress $\boldsymbol{F_u}$ blowing east to west in the tropics (easterlies) and west to east in mid-latitudes (westerlies).

# 1. Define the grid

A 60˚x60˚ box in the North Atlantic, 4000m deep

In [ ]:
Nx = 60
Ny = 60

grid = LatitudeLongitudeGrid(
    size = (Nx, Ny, 1),
    longitude = (-30, 30),
    latitude = (15, 75),
    z = (-500, 0))

Note that we solve the equations in spherical coordinates. This also means that the grid cells get smaller towards to poles. 
- Why do we choose a 500m vs e.g. a 4000m deep ocean? What happens if you choose a shallower ocean?
- As an experiment you can move the box over the Equator and see how the simulated dynamics change if you do
- Are there any problems if we move the box closer to the pole?
- What dynamics do we resolve with only one vertical layer ($Nz = 1$) and why does it depend on gravity?

# 2. Define model components

The free-surface equation for $\eta$ is solved implicitly, allowing for larger time steps and more stability than solving those explicitly. We choose gravity $g$ (m/s) here.

In [ ]:
free_surface = ImplicitFreeSurface(gravitational_acceleration=10)
coriolis = HydrostaticSphericalCoriolis(scheme = EnstrophyConserving())

And the Coriolis parameter depends on the rotation rate of planet Earth. 
- Why do we choose a gravity here that is not 9.81 m/s? And why does that make the dynamics barotropic or baroclinic?
- What is the wave speed of surface gravity waves including Kelvin waves?
- How do you expect the simulated dynamics to change if you increase/decrease the rotation rate, set it to zero or let the planet spin the other way around?

### Forcing: Zonal wind stress

In [ ]:
ρ = 1000    # kg/m^3
τ = 0.2     # N/m^2

@show surface_wind_stress_parameters = (τ₀ = τ/ρ,
                                        Lφ = grid.Ly,
                                        φ₀ = 15)

@inline surface_wind_stress(λ, φ, t, p) = p.τ₀ * cos(2π * (φ - p.φ₀) / p.Lφ)

surface_wind_stress_bc = FluxBoundaryCondition(surface_wind_stress,
                                               parameters = surface_wind_stress_parameters)

- You could let the wind blow in the opposite direction, is that equivalent to spinning the planet the other way around?
- Which part of the dynamics move northward or southward if you change the inflecting point of the cos, controlled here with `φ₀ = 15`?

### Bottom drag

The hydrostatic equations stated above do not have a drag term but here we need to add one in order to have a sink of energy. The wind is continously putting energy into the system (question: always, everywhere?) that energy needs to go somewhere to prevent numerical instabilities from too high velocities. So we add here the following linear drag terms

$$
\begin{align}
\partial_t u &= ... - \mu u \\
\partial_t v &= ... - \mu v
\end{align}
$$

- What happens if you make the linear drag stronger or weaker, what currents are you expecting for a very strong drag?
- What process do these drag terms represent in the real ocean?
- Can you think of a reason why one would want to have a quadratic drag?


In [ ]:
μ = 1 / 10days
@inline u_bottom_drag(i, j, grid, clock, fields, μ) = @inbounds - μ * fields.u[i, j, 1]
@inline v_bottom_drag(i, j, grid, clock, fields, μ) = @inbounds - μ * fields.v[i, j, 1]

u_bottom_drag_bc = FluxBoundaryCondition(u_bottom_drag,
                                         discrete_form = true,
                                         parameters = μ)

v_bottom_drag_bc = FluxBoundaryCondition(v_bottom_drag,
                                         discrete_form = true,
                                         parameters = μ)

u_bcs = FieldBoundaryConditions(top = surface_wind_stress_bc,
                                bottom = u_bottom_drag_bc)

v_bcs = FieldBoundaryConditions(bottom = v_bottom_drag_bc)

### Diffusivity

Drag is not the only sink of energy in this system. For mostly numerical reasons we also need to introduce diffusivity in order to control a cascade of energy towards smaller scales that should not pile up at the grid scale as in reality this cascae would continue towards molecular viscosity. This takes a form of a Laplacian

$$
\begin{align}
\partial_t \mathbf{u} &= ... + \nabla \cdot \nabla (\nu \mathbf{u})
\end{align}
$$


- Look up the viscosity of water, how large is the diffusivity chosen here in comparison?
- And so why would we use the terms "viscosity" and "diffusivity" interchangeably or not?
- Is the resulting forcing up or down the gradient of velocity?

In [ ]:
νh₀ = 5e3 * (60 / grid.Nx)^2
constant_horizontal_diffusivity = HorizontalScalarDiffusivity(ν = νh₀)

# 3. Construct a HydrostaticFreeSurfaceModel

Now we pass on all model components to the `HydrostaticFreeSurfaceModel` and set the ones we want to disable to `nothing`

In [ ]:
model = HydrostaticFreeSurfaceModel(; grid, free_surface, coriolis,
                                    momentum_advection = VectorInvariant(),
                                    boundary_conditions = (u=u_bcs, v=v_bcs),
                                    closure = constant_horizontal_diffusivity,
                                    tracers = nothing,
                                    buoyancy = nothing)

### Choosing a time step

It remains important to choose a time step for the simulation, we will orient ourselves following the wave speed of gravity waves $\sqrt{gH}$. The free-surface solver is implicit so we can use a Courant-Friedrichs-Levy (CFL) condition of larger than 1 but should still roughly scale it with that

In [ ]:
g = model.free_surface.gravitational_acceleration
gravity_wave_speed = sqrt(g * grid.Lz) # hydrostatic (shallow water) gravity wave speed

# Time-scale for gravity wave propagation across the smallest grid cell
wave_propagation_time_scale = min(grid.radius * cosd(maximum(abs, grid.φᵃᶜᵃ)) * deg2rad(grid.Δλᶜᵃᵃ),
                                  grid.radius * deg2rad(grid.Δφᵃᶜᵃ)) / gravity_wave_speed

And so here we decide to take a timestep that is almost 3x as large
- try how much larger you can make the time step before the simulation blows up
- what is the disadvantage of choosing a smaller time step?

Now we wrap our model into a simulation, and decide on the time step and the `stop_time` how long to integrate for

In [ ]:
simulation = Simulation(model,
                        Δt = 1200,
                        stop_time = 365days)

### Defining some feedback and output

The following is to obtain some information about the simulation while it's running and define output

In [ ]:
mutable struct Progress
    interval_start_time :: Float64
end

function (p::Progress)(sim)
    wall_time = (time_ns() - p.interval_start_time) * 1e-9

    @info @sprintf("time: %s, iteration: %d, max(u): %.2e m s⁻¹, wall time: %s",
                   prettytime(sim.model.clock.time),
                   sim.model.clock.iteration,
                   maximum(sim.model.velocities.u),
                   prettytime(wall_time))

    p.interval_start_time = time_ns()

    return nothing
end

simulation.callbacks[:progress] = Callback(Progress(time_ns()), IterationInterval(24*50))
output_fields = merge(model.velocities, (η=model.free_surface.η,))
output_prefix = "barotropic_gyre_Nx$(grid.Nx)_Ny$(grid.Ny)"

simulation.output_writers[:fields] = JLD2Writer(model, output_fields,
                                                schedule = TimeInterval(1day),
                                                filename = output_prefix,
                                                overwrite_existing = true)

# 4. Run the simulation!!!

Now the only left is to run the simulation which will write output at regular intervals

In [ ]:
run!(simulation)

# 5. Visualisation

We can now load in the simulation data and turn the simulation into a video

In [ ]:
using JLD2      # A data format that can write Julia data as-is, alternative to netCDF
using GLMakie   # uses the visualisation library Makie.jl (think Matplotlib if you're coming from Python) with OpenGL backend

In [ ]:
function visualize_barotropic_gyre(filepath)
    file = jldopen(filepath)

    Nx = file["grid/Nx"]    # resolution of simulation
    Ny = file["grid/Ny"]

    # A spherical domain
    grid = LatitudeLongitudeGrid(size = (Nx, Ny, 1),
                                 longitude = (-30, 30),
                                 latitude = (15, 75),
                                 z = (-500, 0))

    iterations = parse.(Int, keys(file["timeseries/t"]))

    # coordinates on u, v, c points (Arakawa C-grid!)
    xu, yu = λnodes(grid, Face()), φnodes(grid, Center())
    xv, yv = λnodes(grid, Center()), φnodes(grid, Face())
    xc, yc = λnodes(grid, Center()), φnodes(grid, Center())

    # turn the iteration into an observable that can be updated
    # which will also update all variables that depend on it!
    iter = Observable(0)

    u = @lift file["timeseries/u/" * string($iter)][:, :, 1]
    v = @lift file["timeseries/v/" * string($iter)][:, :, 1]
    η = @lift file["timeseries/η/" * string($iter)][:, :, 1]

    # create a figure, add some axes and plot some heatmaps into it
    fig = Figure(size=(1100, 400))

    ax1 = Axis(fig[1, 1], title="u")
    heatmap!(ax1, u, colormap=:balance)#, colorrange=(-5, 5))
    hidedecorations!(ax1)
    
    ax2 = Axis(fig[1, 2], title="v")
    heatmap!(ax2, v, colormap=:balance)#, colorrange=(-5, 5))
    hidedecorations!(ax2)
    
    eta_time_title = @lift @sprintf("η, time=%s", prettytime(file["timeseries/t/" * string($iter)]))
    ax3 = Axis(fig[1, 3], title=eta_time_title)
    heatmap!(ax3, η, colormap=:balance)#, colorrange=(-800, 800))
    hidedecorations!(ax3)

    @info "Recording animation..."
    record(fig, output_prefix * ".mp4", iterations, framerate=18) do i
        iter[] = i
    end

    close(file)
    @info "... done."
    
    return nothing
end

Now we creat a video based on the following file

In [ ]:
output_prefix = "barotropic_gyre_Nx$(Nx)_Ny$(Ny)"
filepath = output_prefix * ".jld2"
visualize_barotropic_gyre(filepath)

and look into the the current folder for a `.mp4` video!